In [2]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Lambda
from keras.constraints import maxnorm
from keras.optimizers import adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [3]:
np.random.seed(10)

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [5]:
import tensorflow as tf
from sklearn.utils.random import sample_without_replacement
from sklearn.utils import check_random_state
from scipy.linalg import circulant
from scipy.linalg import hadamard
import random
import math
import tqdm

def gaussian_random_tensor(n_components, n_features, random_state=None):
    rng = check_random_state(random_state)
    components = rng.normal(loc=0.0,
                            scale=1.0 / np.sqrt(n_components),
                            size=(n_components, n_features))
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def hadamard_mat(n_components, n_features):
    had = hadamard(n_features, dtype=complex)
    had = had[np.random.choice(had.shape[0], n_components, replace=False)]
    return had

def diagonal_mat(n_components, n_features):
    components = np.random.rand(n_features)
    components = np.diag(components)
    components = components[0:n_components]
    return components

def circulant_random_tensor(n_components, n_features):
    components = np.random.rand(1, n_features)
    components = circulant(components)
    had = hadamard_mat(n_features, n_features)
    diag1 = diagonal_mat(n_features, n_features)
    diag2 = diagonal_mat(n_features, n_features)
    HD = np.dot(had, diag1)
    DHD = np.dot(diag2, HD)
    components = np.dot(components, DHD)
    components = components[np.random.choice(components.shape[0], n_components, replace=False)]
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def kac_matrix(dim, n):
    maxiter = n * math.log(n)
    for k in tqdm.tqdm(range(1, int(maxiter))):
        i = random.randint(0, n-1)
        j = random.randint(0, n-1)
        while j == i:
            j = random.randint(0, n-1)
        theta = np.random.uniform(0, 2*math.pi)
        B_new = np.identity(n)
        B_new[i, i] = math.sin(theta)
        B_new[i, j] = math.cos(theta)
        B_new[j, i] = -math.cos(theta)
        B_new[j, j] = math.sin(theta)
        if k == 1:
            M = B_new
        else:
            M = np.dot(M, B_new)
    M = M[np.random.choice(M.shape[0], dim, replace=False)]
    return (tf.convert_to_tensor(M, dtype = 'float32'))
          
def project(x, Y):  
    features = K.int_shape(x)[1] 
    x = tf.transpose(x)
    # Y = tf.sign(Y) # use this if it is doing the sign BEFORE (we think right, he supposedly said wrong)
    X_new = K.dot(Y, x)
    X_new = tf.transpose(X_new)
    X_new = tf.sign(X_new) # use this if it is doing the sign AFTER (we think wrong, he says right)
    return X_new

In [ ]:
dims = [784, 392, 196, 98, 49]
import time
accuracy = []
size = []
times = []
epochs = 10

for d in dims:
    start = time.time()
    
    if d != 1568:
        Y = gaussian_random_tensor(d, 1568) 
        # Y = kac_matrix(d, 1568) 
        # Y = circulant_random_tensor(d, 1568) 

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(1,28,28), kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    if d != 1568:
        model.add(Lambda (lambda x: project(x, Y)))
        
    model.add(Dense(50, activation='relu', kernel_constraint=maxnorm(3)))
    
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    h = model.fit(X_train, Y_train, batch_size=32, epochs=epochs, verbose=1)
    
    a = model.evaluate(X_test, Y_test, verbose=0)[1]
    end = time.time()
    t = end - start
    accuracy.append(a)
    size.append(d)
    times.append(t)
    
import pandas as pd
df = pd.DataFrame({'Size': size, 'Accuracy': accuracy, 'Runtime': times})


Epoch 1/10
 1184/60000 [..............................] - ETA: 267s - loss: 1.5810 - acc: 0.4823

In [19]:
df['time_saved'] = (max(df['Runtime']) - df['Runtime']) / max(df['Runtime'])
df['acc_lost'] = (max(df['Accuracy']) - df['Accuracy'])
df.head()


,Accuracy,Runtime,Size,time_saved,acc_lost
0,0.9891,5043.912917,4608,0.260923,0.0019
1,0.9910,6824.607547,3456,0.000000,0.0000
2,0.9867,4590.439886,2304,0.327369,0.0043
3,0.9884,3224.516772,1152,0.527516,0.0026
4,0.9893,2847.549422,576,0.582753,0.0017
